In [34]:
import torch
from torch import nn

In [77]:
class Res_LSTM(nn.Module):
    def __init__(self,dimention,factor_num, sequence):
        super(Res_LSTM, self).__init__()
        self.factor_num = factor_num  # 108
        self.sequence = sequence  #
        self.dimention = dimention 
        # Layer
        self.bn1 = torch.nn.BatchNorm1d(self.dimention * self.factor_num * self.sequence)
        self.bn2 = torch.nn.BatchNorm1d(self.factor_num * 2 * self.sequence)
        self.bn3 = torch.nn.BatchNorm1d(self.factor_num * 2)
        
        
        self.lstm = nn.LSTM(self.factor_num, self.factor_num, 2, batch_first=True, bidirectional=True,
                            dropout=0.2)
        self.TransformerLayer1 = nn.TransformerEncoderLayer(d_model=self.factor_num, nhead=2,batch_first=True)
        self.TransformerLayer2 = nn.TransformerEncoderLayer(d_model=self.factor_num, nhead=2,batch_first=True)
        self.TransformerLayer3 = nn.TransformerEncoderLayer(d_model=self.factor_num, nhead=2,batch_first=True)
        self.TransformerLayer4 = nn.TransformerEncoderLayer(d_model=self.factor_num, nhead=2,batch_first=True)
        
        self.dropout = nn.Dropout(0.2)
        self.LeakyReLU = nn.LeakyReLU()
        self.out = nn.Linear(self.factor_num * 2, 1)

    def forward(self, x):
        x = self.norm_1(x)
        x = torch.transpose(x, 0, 1)  # x.shape: torch.Size([3, 512, 20, 108])

        final, (hn, cn) = self.lstm(x[0])  # torch.Size([512, 20, 128])
        
        # Encoder 1
        encoder1_1 = self.TransformerLayer1(x[1])
        encoder1_2 = self.TransformerLayer2(x[1])
        encoder1 = torch.cat([encoder1_1,encoder1_2],dim=2)
        final = self.skip_connection(final,encoder1)

        # Encoder 1
        encoder2_1 = self.TransformerLayer3(x[2])
        encoder2_2 = self.TransformerLayer4(x[2])
        encoder2 = torch.cat([encoder2_1,encoder2_2],dim=2)
        final = self.skip_connection(final,encoder2)
        
        
        x, _ = self.attention_net(final, hn)
        x = self.bn3(x)
        x = self.LeakyReLU(x)
        x = self.dropout(x)
        y_pred = self.out(x)
        return y_pred

    def norm_1(self, x):
        batch_num, original_shape = x.shape[0], x.shape
        x = x.reshape(batch_num, -1)
        x = self.bn1(x)
        x = x.reshape(original_shape)
        return x

    def norm_2(self, x):
        batch_num, original_shape = x.shape[0], x.shape
        x = x.reshape(batch_num, -1)
        x = self.bn2(x)
        x = x.reshape(original_shape)
        return x

    def skip_connection(self, origin, add):
        return self.norm_2(origin + add) # torch.Size([1024, 20, 216])

    def attention_net(self, lstm_output, final_state):
        # lstm_output : [batch_size, n_step, n_hidden * num_directions(=2)], F matrix
        # final_state : [num_layers(=1) * num_directions(=2), batch_size, n_hidden]

        hidden = torch.cat((final_state[0], final_state[1]), dim=1).unsqueeze(
            2)  # hidden : [batch_size, n_hidden * num_directions(=2), n_layer(=1)]
        attn_weights = torch.bmm(lstm_output, hidden).squeeze(2)  # [batch_size,sequence]
        attn_weights = torch.nn.functional.softmax(attn_weights, 1)  # [batch_size,sequence]   # torch.Size([512, 20])
        # context: [batch_size, n_hidden * num_directions(=2)]
        output = torch.bmm(lstm_output.transpose(1, 2), attn_weights.unsqueeze(2)).squeeze(
            2)  # [batch_size, n_hidden * num_directions(=2)]
        return output, attn_weights

In [52]:
import os
from os import walk
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
import torch.utils.data as Data
import torch.optim as optim
from torch.autograd import Variable
from progressbar import ProgressBar
from tqdm import tqdm
import time
import multiprocessing as mp


trainx.shape:  torch.Size([10000, 3, 20, 108])


In [53]:
trainx = torch.randn(10000,3,20,108)
# trainx = torch.randn(10000,20,108)
trainy = torch.randn(10000,1)
print("trainx.shape: " , trainx.shape)
train_dataset = Data.TensorDataset(trainx, trainy)
batch_size = 1024
train_loader = Data.DataLoader(
    dataset=train_dataset,
    batch_size=batch_size,
    shuffle=True,
    num_workers=16,
    pin_memory=True
)

In [78]:
# alphanet = AlphaNet_LSTM_V1(108,20,64,attention=True)
alphanet = Res_LSTM(3,108,20)
alphanet = alphanet.cuda()
# alphanet = torch.nn.parallel.DataParallel(alphanet)
print(alphanet)
total_length = trainx.shape[0]
LR = 0.01
loss_function = nn.MSELoss().cuda()
optimizer = optim.Adam(alphanet.parameters(), lr=LR)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=2,gamma = 0.5)
epoch_num = 20
loss_list = []

min_loss = float("inf")
for epoch in tqdm(range(epoch_num)):
    total_loss = 0
    for _, (inputs, outputs) in enumerate(train_loader):
        inputs = Variable(inputs).float().cuda()
        outputs = Variable(outputs).float().cuda()
        optimizer.zero_grad() # noticed:  the grad return to zero before starting the loop
        
        # forward + backward +update
        pred = alphanet(inputs.cuda())
        pred = pred.cuda()
        loss = loss_function(pred, outputs)
        loss.backward()
        optimizer.step()
        
#         lr_list.append(optimizer.state_dict()['param_groups'][0]['lr'])
        total_loss += loss.item()
    total_loss = total_loss * batch_size / total_length
    print('Epoch: ', epoch + 1, ' loss: ', total_loss)
    loss_list.append(total_loss)

  0%|          | 0/20 [00:00<?, ?it/s]

Res_LSTM(
  (bn1): BatchNorm1d(6480, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn2): BatchNorm1d(4320, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (bn3): BatchNorm1d(216, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lstm): LSTM(108, 108, num_layers=2, batch_first=True, dropout=0.2, bidirectional=True)
  (TransformerLayer1): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=108, out_features=108, bias=True)
    )
    (linear1): Linear(in_features=108, out_features=2048, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=2048, out_features=108, bias=True)
    (norm1): LayerNorm((108,), eps=1e-05, elementwise_affine=True)
    (norm2): LayerNorm((108,), eps=1e-05, elementwise_affine=True)
    (dropout1): Dropout(p=0.1, inplace=False)
    (dropout2): Dropout(p=0.1, inplace=False)
  )
  (TransformerLayer2): Trans

  5%|▌         | 1/20 [00:03<01:09,  3.65s/it]

Epoch:  1  loss:  1.2155889526367187


 10%|█         | 2/20 [00:07<01:06,  3.68s/it]

...

Epoch:  18  loss:  0.10821964950561523


 95%|█████████▌| 19/20 [01:10<00:03,  3.77s/it]

Epoch:  19  loss:  0.09965411376953125


100%|██████████| 20/20 [01:13<00:00,  3.69s/it]

Epoch:  20  loss:  0.09514836730957031


In [33]:
alphanet = AlphaNet_LSTM_V1(108,20,64,attention=True,transformer = True)
# alphanet = Transformer_LSTM(3,108,20,64)
alphanet = alphanet.cuda()
# alphanet = torch.nn.parallel.DataParallel(alphanet)
print(alphanet)
total_length = trainx.shape[0]
LR = 0.01
loss_function = nn.MSELoss().cuda()
optimizer = optim.Adam(alphanet.parameters(), lr=LR)
# scheduler = torch.optim.lr_scheduler.StepLR(optimizer,step_size=2,gamma = 0.5)
epoch_num = 50
loss_list = []

min_loss = float("inf")
for epoch in tqdm(range(epoch_num)):
    total_loss = 0
    for _, (inputs, outputs) in enumerate(train_loader):
        inputs = Variable(inputs).float().cuda()
        outputs = Variable(outputs).float().cuda()
        optimizer.zero_grad() # noticed:  the grad return to zero before starting the loop
        
        # forward + backward +update
        pred = alphanet(inputs.cuda())
        pred = pred.cuda()
        loss = loss_function(pred, outputs)
        loss.backward()
        optimizer.step()
        
#         lr_list.append(optimizer.state_dict()['param_groups'][0]['lr'])
        total_loss += loss.item()
    total_loss = total_loss * batch_size / total_length
    print('Epoch: ', epoch + 1, ' loss: ', total_loss)
    loss_list.append(total_loss)

  0%|          | 0/50 [00:00<?, ?it/s]

AlphaNet_LSTM_V1(
  (batch): BatchNorm1d(2160, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (lstm): LSTM(108, 64, num_layers=3, batch_first=True, dropout=0.2, bidirectional=True)
  (lstm2): LSTM(128, 32, num_layers=3, batch_first=True, dropout=0.2, bidirectional=True)
  (batch2): BatchNorm1d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (batch3): BatchNorm1d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (dropout): Dropout(p=0.2, inplace=False)
  (relu): ReLU()
  (LeakyReLU): LeakyReLU(negative_slope=0.01)
  (out): Linear(in_features=64, out_features=1, bias=True)
  (TransformerLayer): TransformerEncoderLayer(
    (self_attn): MultiheadAttention(
      (out_proj): NonDynamicallyQuantizableLinear(in_features=128, out_features=128, bias=True)
    )
    (linear1): Linear(in_features=128, out_features=2048, bias=True)
    (dropout): Dropout(p=0.1, inplace=False)
    (linear2): Linear(in_features=2048, out_features=128, bi

  2%|▏         | 1/50 [00:02<01:51,  2.28s/it]

Epoch:  1  loss:  1.1155769775390625


  4%|▍         | 2/50 [00:04<01:48,  2.26s/it]

...

Epoch:  48  loss:  1.0418911193847655


 98%|█████████▊| 49/50 [01:58<00:02,  2.43s/it]

Epoch:  49  loss:  1.0430485595703125


100%|██████████| 50/50 [02:01<00:00,  2.42s/it]

Epoch:  50  loss:  1.0408672729492188
